using code from github: 

Skin Cancer Object Detection Data Preperation/mask-to-bbox-yolo.ipynb

made modifications to run it locally

In [ ]:
import os

# Specify the directory path you want to create
directory = '/kaggle/working/Labels'

# Create the directory
os.makedirs(directory)

In [ ]:
import numpy as np
import cv2
import os
import pandas as pd

def mask_to_bounding_box(image_path):
    df = pd.read_csv("/kaggle/input/skin-cancer-lesions-segmentation/data/metadata.csv")
    # Read the image using OpenCV
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Resize the image to 256x256
    # resized_image = cv2.resize(image, (256, 256))

    # Convert the resized image to numpy array
    numpy_array = np.array(image)

    # Find non-zero coordinates in the mask
    non_zero_indices = np.nonzero(numpy_array)

    # Calculate bounding box coordinates
    min_y = np.min(non_zero_indices[0])
    max_y = np.max(non_zero_indices[0])
    min_x = np.min(non_zero_indices[1])
    max_x = np.max(non_zero_indices[1])

    # Calculate width and height
    width = max_x - min_x + 1
    height = max_y - min_y + 1

    # Normalize the coordinates and dimensions
    img_height, img_width = image.shape[:2]
    min_x_norm = min_x / width
    min_y_norm = min_y / height
    width_norm = width / img_width  # Normalized width
    height_norm = height / img_height  # Normalized height

    # Extract the image name
    img_name = os.path.basename(image_path).split('.')[0]

    # Class ID
    mapping = {'MEL': 0, 'NV': 1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}
    row = df[df['image'] == img_name]
    clss = row.columns[(row == 1).iloc[0]][0]
    class_ID = mapping[clss]

    # Create a string with bounding box details
    bounding_box_details = f"{class_ID} {min_x_norm} {min_y_norm} {width_norm} {height_norm}\n"
    
    #Output_file
    output_file = f"/kaggle/working/Labels/{img_name}.txt"
    # Write bounding box details to the output file
    with open(output_file, "w") as file:
        file.write(bounding_box_details)


# Example usage:
img_path = "/kaggle/input/skin-cancer-lesions-segmentation/data/masks/ISIC_0024311.png"
bounding_box = mask_to_bounding_box(img_path)

In [ ]:
# Specify the directory path
masks_directory = "/kaggle/input/skin-cancer-lesions-segmentation/data/masks"

# Loop through all the files in the directory
for filename in os.listdir(masks_directory):
    # Check if the current file is a regular file
    file_path = os.path.join(masks_directory, filename)
    mask_to_bounding_box(file_path)